In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler, PowerTransformer,QuantileTransformer,RobustScaler
from sklearn.model_selection import GridSearchCV, cross_val_score
from catboost import CatBoostRegressor

In [4]:
base_path = os.path.join("/", "Volumes", "Extreme SSD", "ShellAi")
train_path = os.path.join(base_path, "train.csv")
test_path = os.path.join(base_path, "test.csv")

# Load the CSV files
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

In [6]:
# Creating  weighted columns
base_features = [col for col in train_df.columns if col not in [f"BlendProperty{i}" for i in range(1, 11)]]
targets = [f"BlendProperty{i}" for i in range(1, 11)]
weighted_data = {}
for i in range(1, 6):
    for j in range(1, 11):
        prop_col = f"Component{i}_Property{j}"
        frac_col = f"Component{i}_fraction"
        weighted_col = f"Weighted_Component{i}_Property{j}"
        weighted_data[weighted_col] = train_df[prop_col] * train_df[frac_col]

# Combine
data_with_weighted = pd.concat([train_df[base_features], pd.DataFrame(weighted_data)], axis=1)
data_with_weighted = data_with_weighted.copy()  # De-fragmented copy

In [8]:
scaler = RobustScaler()
scaled_features = scaler.fit_transform(data_with_weighted)
scaled_df_features = pd.DataFrame(scaled_features, columns=data_with_weighted.columns)


pt = PowerTransformer(method='yeo-johnson')
transformed_targets_df = pd.DataFrame(pt.fit_transform(train_df[targets]), columns=targets, index=train_df.index)

# Initial feature selection using Random Forest feature importance
top_features = {}
for target in targets:
    X = scaled_df_features
    y = transformed_targets_df[target]
    rf_initial = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_initial.fit(X, y)
    feature_importance = pd.Series(rf_initial.feature_importances_, index=scaled_df_features.columns).sort_values(ascending=False)
    top_features[target] = feature_importance.head(20).index.tolist()

# GridSearchCV
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20,]

}


In [32]:
warnings.filterwarnings("ignore",category=UserWarning)
for target in targets:
    print(f"\n🔹 Target: {target}")

    X = scaled_df_features[top_features[target]]
    y = transformed_targets_df[target]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = CatBoostRegressor(verbose=0, random_state=42)
    grid_search = GridSearchCV(model, param_grid, cv=5,
                               scoring='neg_mean_absolute_percentage_error', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_
    print(f"Best params: {grid_search.best_params_}")

    # CV MAPE on transformed targets
    cv_mape = -cross_val_score(best_model, X_train, y_train, cv=5,
                               scoring='neg_mean_absolute_percentage_error').mean() * 100
    print(f"Cross-validated MAPE (transformed): {cv_mape:.2f}%")

    # Predict on test set
    y_pred = best_model.predict(X_test)

    # Inverse transform for original scale MAPE
    y_test_full = pd.DataFrame(np.column_stack([y_test] + [np.zeros(len(y_test))]*(len(targets)-1)),
                               columns=targets, index=X_test.index)
    y_pred_full = pd.DataFrame(np.column_stack([y_pred] + [np.zeros(len(y_pred))]*(len(targets)-1)),
                               columns=targets, index=X_test.index)

    y_test_inv = pt.inverse_transform(y_test_full)[targets.index(target)]
    y_pred_inv = pt.inverse_transform(y_pred_full)[targets.index(target)]

    # Safe MAPE
    mask = y_test_inv != 0
    test_mape = np.mean(np.abs((y_test_inv[mask] - y_pred_inv[mask]) / y_test_inv[mask]) * 100) if mask.any() else 0.0
    print(f"Test MAPE (original scale): {test_mape:.2f}%")


🔹 Target: BlendProperty1
Best params: {'max_depth': 10, 'n_estimators': 200}
Cross-validated MAPE (transformed): 77.03%
Test MAPE (original scale): 0.70%

🔹 Target: BlendProperty2
Best params: {'max_depth': 10, 'n_estimators': 100}
Cross-validated MAPE (transformed): 74.20%
Test MAPE (original scale): 1.27%

🔹 Target: BlendProperty3
Best params: {'max_depth': 10, 'n_estimators': 200}
Cross-validated MAPE (transformed): 69.74%
Test MAPE (original scale): 0.38%

🔹 Target: BlendProperty4
Best params: {'max_depth': 10, 'n_estimators': 200}
Cross-validated MAPE (transformed): 81.47%
Test MAPE (original scale): 34.82%

🔹 Target: BlendProperty5
Best params: {'max_depth': 10, 'n_estimators': 100}
Cross-validated MAPE (transformed): 57.89%
Test MAPE (original scale): 0.71%

🔹 Target: BlendProperty6
Best params: {'max_depth': 10, 'n_estimators': 200}
Cross-validated MAPE (transformed): 74.37%
Test MAPE (original scale): 1.51%

🔹 Target: BlendProperty7
Best params: {'max_depth': 10, 'n_estimator

In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.preprocessing import PowerTransformer, QuantileTransformer
from catboost import CatBoostRegressor
import warnings
import time # Import time module to measure execution time

warnings.filterwarnings("ignore", category=UserWarning)

# --- ASSUMED PRE-EXISTING DATA (same as before for runnable example) ---
num_samples = 1000
num_features = 50
targets = [f'BlendProperty{i}' for i in range(1, 11)]

base_features = pd.DataFrame(np.random.rand(num_samples, num_features) * 100, columns=[f'feature_{i}' for i in range(num_features)])
scaled_df_features = base_features

top_features = {}
for target in targets:
    num_top = np.random.randint(5, 15)
    top_features[target] = np.random.choice(base_features.columns, num_top, replace=False).tolist()

original_targets_df = pd.DataFrame(index=base_features.index)
for target in targets:
    if target in ['BlendProperty4', 'BlendProperty9']:
        vals = np.random.rand(num_samples) * 50
        vals[np.random.choice(num_samples, int(num_samples * 0.1), replace=False)] = 0
        original_targets_df[target] = vals
    else:
        original_targets_df[target] = np.random.rand(num_samples) * 100 + 1
# --- END ASSUMED PRE-EXISTING DATA ---


# Dictionary to store fitted transformers for each target
target_transformers = {}

# --- Hyperparameter Grid for CatBoost (Reduced for faster initial tuning) ---
param_grid = {
    'n_estimators': [200, 500, 800], # Fewer options, more manageable
    'max_depth': [8, 10, 12],       # Fewer options
    'learning_rate': [0.03, 0.05, 0.08], # Tighter range
    'l2_leaf_reg': [3, 5, 7],           # Fewer options
    'subsample': [0.7, 0.9],            # Fewer options
    'colsample_bylevel': [0.7, 0.9],    # Fewer options
    'min_data_in_leaf': [3, 10],        # Fewer options
}

# Parameters for RandomizedSearchCV (Reduced iterations and CV folds for speed)
n_iterations = 25 # SIGNIFICANTLY REDUCED: Try 20-30 for initial runs
cv_folds_tuning = 3 # REDUCED CV folds for the tuning phase
cv_folds_evaluation = 5 # Use 5 folds for final cross_val_score if time permits

print(f"Starting model training with {n_iterations} iterations and {cv_folds_tuning}-fold CV per target.")
print(f"Total estimated CatBoost fits: {len(targets)} targets * {n_iterations} iter * {cv_folds_tuning} cv = {len(targets) * n_iterations * cv_folds_tuning}")

# --- Main Loop for Each Target ---
for i, target in enumerate(targets):
    start_time_target = time.time() # Start timer for current target
    print(f"\n🔹 Target: {target}")

    # --- 1. Target-Specific Data Preparation & Transformation ---
    X = scaled_df_features[top_features[target]]
    y_original = original_targets_df[target]

    if target in ['BlendProperty4', 'BlendProperty9']:
        transformer = QuantileTransformer(output_distribution='normal', random_state=42)
        print(f"   Using QuantileTransformer for {target}")
    else:
        transformer = PowerTransformer(method='yeo-johnson')
        print(f"   Using PowerTransformer (Yeo-Johnson) for {target}")

    y_transformed = transformer.fit_transform(y_original.values.reshape(-1, 1)).flatten()
    target_transformers[target] = transformer

    X_train, X_test, y_train_transformed, y_test_transformed = train_test_split(
        X, y_transformed, test_size=0.2, random_state=42
    )
    # y_test_original is needed for original scale MAPE
    _, _, _, y_test_original = train_test_split(
        X, y_original, test_size=0.2, random_state=42
    )

    # --- 2. Model Training with RandomizedSearchCV ---
    model = CatBoostRegressor(verbose=0, random_state=42)

    random_search = RandomizedSearchCV(
        model,
        param_distributions=param_grid,
        n_iter=n_iterations,
        cv=cv_folds_tuning, # Use reduced CV for tuning
        scoring='neg_mean_absolute_percentage_error',
        n_jobs=-1,
        random_state=42,
        verbose=0
    )
    random_search.fit(X_train, y_train_transformed)

    best_model = random_search.best_estimator_
    print(f"Best params: {random_search.best_params_}")

    # --- 3. Evaluation on Transformed Data (Cross-validated) ---
    cv_mape_transformed = -cross_val_score(
        best_model,
        X_train,
        y_train_transformed,
        cv=cv_folds_evaluation, # Use full CV for a more robust estimate here
        scoring='neg_mean_absolute_percentage_error'
    ).mean() * 100
    print(f"Cross-validated MAPE (transformed): {cv_mape_transformed:.2f}%")

    # --- 4. Prediction and Inverse Transformation for Original Scale MAPE ---
    y_pred_transformed = best_model.predict(X_test)

    y_test_inv = target_transformers[target].inverse_transform(y_test_transformed.reshape(-1, 1)).flatten()
    y_pred_inv = target_transformers[target].inverse_transform(y_pred_transformed.reshape(-1, 1)).flatten()

    # --- 5. Safe MAPE Calculation on Original Scale ---
    epsilon = 1e-6
    mask = y_test_inv != 0

    if mask.any():
        test_mape_original = np.mean(np.abs((y_test_inv[mask] - y_pred_inv[mask]) / (y_test_inv[mask] + epsilon)) * 100)
    else:
        test_mape_original = 0.0

    print(f"Test MAPE (original scale): {test_mape_original:.2f}%")

    # --- Optional: Detailed Analysis for BlendProperty4 and BlendProperty9 ---
    if target in ['BlendProperty4', 'BlendProperty9']:
        print(f"   Consider a two-stage model for {target} if many zeros/near-zeros are present.")
        print("   Examine feature importance specific to this target's large errors.")
        # Add your plotting and specific error analysis here if you want.
        # Example plotting code would go here:
        # import matplotlib.pyplot as plt
        # plt.figure(figsize=(10, 5))
        # plt.subplot(1, 2, 1)
        # y_original.hist(bins=50, alpha=0.7, color='skyblue')
        # plt.title(f'Original Distribution of {target}')
        # plt.subplot(1, 2, 2)
        # pd.Series(y_transformed).hist(bins=50, alpha=0.7, color='lightcoral')
        # plt.title(f'Transformed Distribution of {target}')
        # plt.tight_layout()
        # plt.show()


    end_time_target = time.time() # End timer for current target
    print(f"Time taken for {target}: {end_time_target - start_time_target:.2f} seconds")

# --- AFTER THE LOOP: ENSEMBLE/FINAL PREDICTION STRATEGY ---
print("\n--- Model Training Completed for all Targets ---")
print("\nNext Steps for Further Optimization:")
print("1. After finding good general hyperparameter ranges with reduced iterations/CV, you can increase `n_iterations` for a more thorough search or focus on specific promising ranges.")
print("2. Implement Ensemble Methods (e.g., averaging predictions from multiple best models, or stacking).")
print("3. Deep dive into feature engineering, especially for 'BlendProperty4' and 'BlendProperty9'.")
print("4. Consider a two-stage model (classification for zero/non-zero, then regression) for targets with many zeros.")
print("5. Experiment with different random seeds for training to check stability and average results.")
print("6. If still too slow, consider LightGBM or XGBoost which are often faster than CatBoost, though CatBoost has advantages for categorical features.")



Starting model training with 25 iterations and 3-fold CV per target.
Total estimated CatBoost fits: 10 targets * 25 iter * 3 cv = 750

🔹 Target: BlendProperty1
   Using PowerTransformer (Yeo-Johnson) for BlendProperty1
Best params: {'subsample': 0.9, 'n_estimators': 200, 'min_data_in_leaf': 10, 'max_depth': 12, 'learning_rate': 0.03, 'l2_leaf_reg': 7, 'colsample_bylevel': 0.7}
Cross-validated MAPE (transformed): 457.11%
Test MAPE (original scale): 152.24%
Time taken for BlendProperty1: 478.28 seconds

🔹 Target: BlendProperty2
   Using PowerTransformer (Yeo-Johnson) for BlendProperty2


KeyboardInterrupt: 

In [42]:
!pip install 'optuna-integration[catboost]'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.9/98.9 kB 376.2 kB/s eta 0:00:00a 0:00:01


In [46]:
import optuna
from catboost import CatBoostRegressor, Pool
from optuna.integration import CatBoostPruningCallback  



In [113]:
import optuna
from sklearn.metrics import mean_absolute_percentage_error
from catboost import CatBoostRegressor

# Suppress warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
best_models = {}  # ← define this before the loop

# Inside the for-loop after training
 # ← define this before the loop

# Inside the for-loop after training


# Loop through each target
for target in targets:
    print(f"\n🔹 Optimizing Target: {target}")

    X = scaled_df_features[top_features[target]]
    y = transformed_targets_df[target]

    # Split for Optuna
    X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)

    # Define objective function
    def objective(trial):
        params = {
            "iterations": trial.suggest_int("iterations", 100, 1000),
            "depth": trial.suggest_int("depth", 4, 10),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
            "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1, 10),
            "random_strength": trial.suggest_float("random_strength", 1e-9, 10.0),
            "loss_function": "MAPE",
            "verbose": 0
        }
        model = CatBoostRegressor(**params)
        model.fit(X_train, y_train)
        preds = model.predict(X_valid)
        return mean_absolute_percentage_error(y_valid, preds)

    # Run Optuna
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=40, show_progress_bar=True)

    best_params = study.best_params
    print(f"✅ Best params for {target}: {best_params}")

    # Train on full training data
    best_model = CatBoostRegressor(**best_params)
    best_model.fit(X_train_full, y_train_full)

    # Predict on test
    y_pred = best_model.predict(X_test)

    # Inverse transform for original scale MAPE
    y_test_full = pd.DataFrame(np.column_stack([y_test] + [np.zeros(len(y_test))]*(len(targets)-1)),
                               columns=targets, index=X_test.index)
    y_pred_full = pd.DataFrame(np.column_stack([y_pred] + [np.zeros(len(y_pred))]*(len(targets)-1)),
                               columns=targets, index=X_test.index)

    y_test_inv = pt.inverse_transform(y_test_full)[targets.index(target)]
    y_pred_inv = pt.inverse_transform(y_pred_full)[targets.index(target)]

    # Safe MAPE
    mask = y_test_inv != 0
    test_mape = np.mean(np.abs((y_test_inv[mask] - y_pred_inv[mask]) / y_test_inv[mask]) * 100) if mask.any() else 0.0
    best_models[target] = best_model
    print(f"📉 Test MAPE (original scale): {test_mape:.2f}%")



[I 2025-07-18 00:07:45,094] A new study created in memory with name: no-name-ef6cf9ed-e5ec-42cf-b5a3-9c458ff62d33



🔹 Optimizing Target: BlendProperty1


  0%|          | 0/40 [00:00<?, ?it/s]

[I 2025-07-18 00:07:46,658] Trial 0 finished with value: 0.5067652493825582 and parameters: {'iterations': 321, 'depth': 6, 'learning_rate': 0.10274099844938712, 'l2_leaf_reg': 9.111231737434888, 'random_strength': 3.598139456724781}. Best is trial 0 with value: 0.5067652493825582.
[I 2025-07-18 00:07:48,044] Trial 1 finished with value: 0.7275947526556562 and parameters: {'iterations': 970, 'depth': 7, 'learning_rate': 0.20145532988306608, 'l2_leaf_reg': 7.9132491596575, 'random_strength': 6.700756814461257}. Best is trial 0 with value: 0.5067652493825582.
[I 2025-07-18 00:07:50,021] Trial 2 finished with value: 0.42561622690752865 and parameters: {'iterations': 853, 'depth': 8, 'learning_rate': 0.0390034837581032, 'l2_leaf_reg': 6.552136426699168, 'random_strength': 1.5558232006048653}. Best is trial 2 with value: 0.42561622690752865.
[I 2025-07-18 00:07:50,613] Trial 3 finished with value: 0.4747965924882081 and parameters: {'iterations': 987, 'depth': 5, 'learning_rate': 0.14687763

[I 2025-07-18 00:08:21,119] A new study created in memory with name: no-name-cd933458-0ba9-4118-911c-58609b88b1a8


...

[I 2025-07-18 00:13:41,258] Trial 0 finished with value: 0.9514721858253481 and parameters: {'iterations': 914, 'depth': 7, 'learning_rate': 0.2316833145524914, 'l2_leaf_reg': 4.256453898201748, 'random_strength': 4.291325044198839}. Best is trial 0 with value: 0.9514721858253481.
[I 2025-07-18 00:13:46,002] Trial 1 finished with value: 0.8359393828652275 and parameters: {'iterations': 515, 'depth': 10, 'learning_rate': 0.05531472168798561, 'l2_leaf_reg': 9.083556181783651, 'random_strength': 5.372816714729701}. Best is trial 1 with value: 0.8359393828652275.
[I 2025-07-18 00:13:46,426] Trial 2 finished with value: 0.9221290556513667 and parameters: {'iterations': 389, 'depth': 6, 'learning_rate': 0.25847407219024326, 'l2_leaf_reg': 1.2701358288749107, 'random_strength': 8.67357723432813}. Best is trial 1 with value: 0.8359393828652275.
[I 2025-07-18 00:13:51,947] Trial 3 finished with value: 1.2391882170341144 and parameters: {'iterations': 609, 'depth': 10, 'learning_rate': 0.2882888

[I 2025-07-18 00:14:33,253] A new study created in memory with name: no-name-a4b0226e-5932-4d7d-ae62-b99b05a2409e


399:	learn: 0.2280485	total: 187ms	remaining: 153ms
400:	learn: 0.2279281	total: 187ms	remaining: 153ms
401:	learn: 0.2275699	total: 188ms	remaining: 152ms
402:	learn: 0.2272055	total: 188ms	remaining: 152ms
403:	learn: 0.2269156	total: 189ms	remaining: 151ms
404:	learn: 0.2264415	total: 189ms	remaining: 151ms
405:	learn: 0.2262029	total: 190ms	remaining: 150ms
406:	learn: 0.2258259	total: 190ms	remaining: 150ms
407:	learn: 0.2255015	total: 190ms	remaining: 149ms
408:	learn: 0.2251538	total: 191ms	remaining: 149ms
409:	learn: 0.2247886	total: 191ms	remaining: 148ms
410:	learn: 0.2246702	total: 192ms	remaining: 148ms
411:	learn: 0.2245353	total: 192ms	remaining: 148ms
412:	learn: 0.2242154	total: 193ms	remaining: 147ms
413:	learn: 0.2238297	total: 193ms	remaining: 147ms
414:	learn: 0.2234824	total: 194ms	remaining: 146ms
415:	learn: 0.2232002	total: 194ms	remaining: 146ms
416:	learn: 0.2230987	total: 195ms	remaining: 145ms
417:	learn: 0.2228007	total: 195ms	remaining: 145ms
418:	learn: 

  0%|          | 0/40 [00:00<?, ?it/s]

[I 2025-07-18 00:14:37,960] Trial 0 finished with value: 1.532469490182545 and parameters: {'iterations': 538, 'depth': 10, 'learning_rate': 0.2129254217012503, 'l2_leaf_reg': 9.455917748499916, 'random_strength': 5.310116295466133}. Best is trial 0 with value: 1.532469490182545.
[I 2025-07-18 00:14:38,594] Trial 1 finished with value: 0.445942040474338 and parameters: {'iterations': 901, 'depth': 5, 'learning_rate': 0.11327330446951976, 'l2_leaf_reg': 9.693602854326658, 'random_strength': 3.5302100035235493}. Best is trial 1 with value: 0.445942040474338.
[I 2025-07-18 00:14:42,946] Trial 2 finished with value: 0.8940885018506355 and parameters: {'iterations': 882, 'depth': 9, 'learning_rate': 0.030638528661154844, 'l2_leaf_reg': 5.906287150560765, 'random_strength': 9.261862680183807}. Best is trial 1 with value: 0.445942040474338.
[I 2025-07-18 00:14:43,144] Trial 3 finished with value: 0.6818464931273878 and parameters: {'iterations': 379, 'depth': 4, 'learning_rate': 0.28070418770

In [115]:
print("Trained models:", best_models.keys())
missing = [t for t in targets if t not in best_models]
print("Missing models:", missing)

weighted_test = {}
for i in range(1, 6):
    for j in range(1, 11):
        prop_col = f"Component{i}_Property{j}"
        frac_col = f"Component{i}_fraction"
        weighted_col = f"Weighted_Component{i}_Property{j}"
        weighted_test[weighted_col] = test_df[prop_col] * test_df[frac_col]

test_with_weighted = pd.concat([test_df[base_features], pd.DataFrame(weighted_test)], axis=1)
test_scaled = pd.DataFrame(scaler.transform(test_with_weighted), columns=test_with_weighted.columns)

# Predict with Optuna models


Trained models: dict_keys(['BlendProperty1', 'BlendProperty2', 'BlendProperty3', 'BlendProperty4', 'BlendProperty5', 'BlendProperty6', 'BlendProperty7', 'BlendProperty8', 'BlendProperty9', 'BlendProperty10'])
Missing models: []


In [117]:
submission_preds=[]
for target in targets:
    model=best_models[target]
    expected_features=model.feature_names_
    X_test_target=test_scaled[expected_features]
    pred_transformed=model.predict(X_test_target)
    pred_full=pd.DataFrame(np.column_stack([pred_transformed]+[np.zeros(len(pred_transformed))]*(len(targets)-1)),columns=targets,index=test_df.index)
    pred_original=pt.inverse_transform(pred_full)[:,targets.index(target)]
    submission_preds.append(pred_original)

submission=pd.DataFrame(np.column_stack(submission_preds),columns=targets,index=test_df.index)
submission.to_csv("submission4.csv",index=False)
print("Submission file created: submission4.csv")

Submission file created: submission4.csv
